# Data Iterator

In [24]:
import torch
from torchtext.legacy import data
import torchtext 
import numpy as np
from torch.autograd import Variable

In [25]:
import torch.nn as nn

In [26]:
import pandas as pd

In [27]:
torch.cuda.is_available()

False

In [4]:
def nopeak_mask(size, opt):
    np_mask = np.triu(np.ones((1, size, size)),
    k=1).astype('uint8')
    np_mask =  Variable(torch.from_numpy(np_mask) == 0)
    if opt.device == 0:
        np_mask = np_mask.cuda()
    return np_mask

def create_masks(src, trg, opt):
    
    src_mask = (src != opt.src_pad).unsqueeze(-2)

    if trg is not None:
        trg_mask = (trg != opt.trg_pad).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        np_mask = nopeak_mask(size, opt)
        if trg.is_cuda:
            np_mask.cuda()
        trg_mask = trg_mask & np_mask
        
    else:
        trg_mask = None
    return src_mask, trg_mask

# patch on Torchtext's batching process that makes it more efficient
# from http://nlp.seas.harvard.edu/2018/04/03/attention.html#position-wise-feed-forward-networks

class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

global max_src_in_batch, max_tgt_in_batch

def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

# Create Tokenizer

In [5]:
import spacy
import re

In [6]:
class tokenize(object):
    
    def __init__(self, lang):
        self.nlp = spacy.load(lang)
            
    def tokenizer(self, sentence):
        sentence = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sentence))
        sentence = re.sub(r"[ ]+", " ", sentence)
        sentence = re.sub(r"\!+", "!", sentence)
        sentence = re.sub(r"\,+", ",", sentence)
        sentence = re.sub(r"\?+", "?", sentence)
        sentence = sentence.lower()
        return [tok.text for tok in self.nlp.tokenizer(sentence) if tok.text != " "]

In [7]:
lang = "en_core_web_sm"
tk = tokenize(lang)

In [8]:
sent = '"Top-down economics never works," said Obama. "The country does not succeed when just those at the very top are doing well. We succeed when the middle class gets bigger, when it feels greater security."'
sent_tk = tk.tokenizer(sent)

In [9]:
!pip install dill

In [10]:
import os
import dill as pickle

In [11]:
def read_data(opt):
    
    if opt.src_data is not None:
        try:
            opt.src_data = open(opt.src_data).read().strip().split('\n')
        except:
            print("error: '" + opt.src_data + "' file not found")
            quit()
    
    if opt.trg_data is not None:
        try:
            opt.trg_data = open(opt.trg_data).read().strip().split('\n')
        except:
            print("error: '" + opt.trg_data + "' file not found")
            quit()

In [12]:
def create_fields(opt):
    
    #     spacy_langs = ['en', 'fr', 'de', 'es', 'pt', 'it', 'nl']
    #     if opt.src_lang not in spacy_langs:
    #         print('invalid src language: ' + opt.src_lang + 'supported languages : ' + spacy_langs)  
    #     if opt.trg_lang not in spacy_langs:
    #         print('invalid trg language: ' + opt.trg_lang + 'supported languages : ' + spacy_langs)
    
    print("loading spacy tokenizers...")
    
    t_src = tokenize(opt.src_lang)
    t_trg = tokenize(opt.trg_lang)

    TRG = data.Field(lower=True, tokenize=t_trg.tokenizer, init_token='<sos>', eos_token='<eos>')
    SRC = data.Field(lower=True, tokenize=t_src.tokenizer)

    if opt.load_weights is not None:
        try:
            print("loading presaved fields...")
            SRC = pickle.load(open(f'{opt.load_weights}/SRC.pkl', 'rb'))
            TRG = pickle.load(open(f'{opt.load_weights}/TRG.pkl', 'rb'))
        except:
            print("error opening SRC.pkl and TXT.pkl field files, please ensure they are in " + opt.load_weights + "/")
            quit()
        
    return(SRC, TRG)

In [13]:
def create_dataset(opt, SRC, TRG):

    print("creating dataset and iterator... ")

    raw_data = {'src' : [line for line in opt.src_data], 'trg': [line for line in opt.trg_data]}
    df = pd.DataFrame(raw_data, columns=["src", "trg"])
    df.to_csv("temp.csv", index=False)
    
    mask = (df['src'].str.count(' ') < opt.max_strlen) & (df['trg'].str.count(' ') < opt.max_strlen)
    df = df.loc[mask]

    df.to_csv("translate_transformer_temp.csv", index=False)
    
    data_fields = [('src', SRC), ('trg', TRG)]
    train = data.TabularDataset('./translate_transformer_temp.csv', format='csv', fields=data_fields)

    train_iter = MyIterator(train, batch_size=opt.batchsize, device=opt.device,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=True, shuffle=True)
    
    # os.remove('translate_transformer_temp.csv')

    if opt.load_weights is None:
        SRC.build_vocab(train)
        TRG.build_vocab(train)
        if opt.checkpoint > 0:
            try:
                os.mkdir("weights")
            except:
                print("weights folder already exists, run program with -load_weights weights to load them")
                quit()
            pickle.dump(SRC, open('weights/SRC.pkl', 'wb'))
            pickle.dump(TRG, open('weights/TRG.pkl', 'wb'))

    opt.src_pad = SRC.vocab.stoi['<pad>']
    opt.trg_pad = TRG.vocab.stoi['<pad>']

    opt.train_len = get_len(train_iter)

    return train_iter

def get_len(train):

    for i, b in enumerate(train):
        pass
    
    return i

In [48]:
class Opt:
    src_data = "../data/english.txt"
    trg_data = "../data/french.txt"
    src_lang = "en_core_web_sm"
    trg_lang = "fr_core_news_sm"
    no_cuda = True
    load_weights = None
    max_strlen = 80
    batchsize = 32
    checkpoint = 0
    d_model = 512
    n_layers = 6
    heads = 8
    dropout = 0.1
    epochs = 1
    printevery = 5
    lr = 0.0001
    
    

In [49]:
opt = Opt()

In [50]:
torch.cuda.is_available()

False

In [51]:
opt.device = 0 if opt.no_cuda is False else -1
if opt.device == 0:
    assert torch.cuda.is_available()

In [52]:
opt.device

-1

In [53]:
read_data(opt)

In [54]:
len(opt.trg_data)

154883

In [55]:
SRC, TRG = create_fields(opt)

loading spacy tokenizers...


In [56]:
opt.train = create_dataset(opt, SRC, TRG)

creating dataset and iterator... 


The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [57]:
from messy_transformer_correct import Transformer

In [58]:
def get_model(opt, src_vocab, trg_vocab):
    
    assert opt.d_model % opt.heads == 0
    assert opt.dropout < 1

    model = Transformer(src_vocab, trg_vocab, opt.d_model, opt.n_layers, opt.heads, opt.dropout)
       
    if opt.load_weights is not None:
        print("loading pretrained weights...")
        model.load_state_dict(torch.load(f'{opt.load_weights}/model_weights'))
    else:
        for p in model.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p) 
    
    if opt.device == 0:
        model = model.cuda()
    
    return model

In [59]:
model = get_model(opt, len(SRC.vocab), len(TRG.vocab))

In [60]:
opt.optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, betas=(0.9, 0.98), eps=1e-9)

In [61]:
from messy_scheduler import CosineWithRestarts

In [62]:
opt.SGDR = True

In [63]:
if opt.SGDR == True:
    opt.sched = CosineWithRestarts(opt.optimizer, T_max=opt.train_len)

In [64]:
opt.checkpoint = 0

In [65]:
if opt.checkpoint > 0:
    print("model weights will be saved every %d minutes and at end of epoch to directory weights/"%(opt.checkpoint))

In [66]:
opt.floyd = False

In [67]:
import time

In [70]:
import torch.nn.functional as F

In [74]:
def train_model(model, opt):
    
    print("training model...")
    model.train()
    start = time.time()
    if opt.checkpoint > 0:
        cptime = time.time()
                 
    for epoch in range(opt.epochs):

        total_loss = 0
        if opt.floyd is False:
            print("   %dm: epoch %d [%s]  %d%%  loss = %s" %\
            ((time.time() - start)//60, epoch + 1, "".join(' '*20), 0, '...'), end='\r')
        
        if opt.checkpoint > 0:
            torch.save(model.state_dict(), 'weights/model_weights')
                    
        for i, batch in enumerate(opt.train): 

            src = batch.src.transpose(0,1)
            trg = batch.trg.transpose(0,1)
            trg_input = trg[:, :-1]
            src_mask, trg_mask = create_masks(src, trg_input, opt)
            preds = model(src, trg_input, src_mask, trg_mask)
            ys = trg[:, 1:].contiguous().view(-1)
            opt.optimizer.zero_grad()
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=opt.trg_pad)
            loss.backward()
            opt.optimizer.step()
            if opt.SGDR == True: 
                opt.sched.step()
            
            total_loss += loss.item()
            
            p = int(100 * (i + 1) / opt.train_len)
            avg_loss = total_loss/opt.printevery
            if (i + 1) % opt.printevery == 0:
                if opt.floyd is False:
                    print("   %dm: epoch %d [%s%s]  %d%%  loss = %.3f" %\
                    ((time.time() - start)//60, epoch + 1, "".join('#'*(p//5)), "".join(' '*(20-(p//5))), p, avg_loss), end='\r')
            else:
                print("   %dm: epoch %d [%s%s]  %d%%  loss = %.3f" %\
                ((time.time() - start)//60, epoch + 1, "".join('#'*(p//5)), "".join(' '*(20-(p//5))), p, avg_loss))
                total_loss = 0
            
            if opt.checkpoint > 0 and ((time.time()-cptime)//60) // opt.checkpoint >= 1:
                torch.save(model.state_dict(), 'weights/model_weights')
                cptime = time.time()
   
   
        print("%dm: epoch %d [%s%s]  %d%%  loss = %.3f\nepoch %d complete, loss = %.03f" %\
        ((time.time() - start)//60, epoch + 1, "".join('#'*(100//5)), "".join(' '*(20-(100//5))), 100, avg_loss, epoch + 1, avg_loss))



In [75]:
train_model(model, opt)

training model...
   0m: epoch 1 [                    ]  0%  loss = 1.954
   0m: epoch 1 [                    ]  0%  loss = 1.930
   0m: epoch 1 [                    ]  0%  loss = 1.920
   0m: epoch 1 [                    ]  0%  loss = 1.934
   0m: epoch 1 [                    ]  0%  loss = 3.802
   0m: epoch 1 [                    ]  0%  loss = 1.904
   0m: epoch 1 [                    ]  0%  loss = 1.885
   0m: epoch 1 [                    ]  0%  loss = 1.845
   0m: epoch 1 [                    ]  0%  loss = 3.723
   0m: epoch 1 [                    ]  0%  loss = 1.830
   0m: epoch 1 [                    ]  0%  loss = 1.889
   0m: epoch 1 [                    ]  0%  loss = 1.791
   0m: epoch 1 [                    ]  0%  loss = 3.684
   0m: epoch 1 [                    ]  0%  loss = 1.821
   0m: epoch 1 [                    ]  0%  loss = 1.810
   0m: epoch 1 [                    ]  0%  loss = 1.812
   0m: epoch 1 [                    ]  0%  loss = 3.489
   0m: epoch 1 [              

KeyboardInterrupt: 